# Data Exploration and cleaning

In [1]:
import numpy as np
import pandas as pd

In [2]:
# import load_data function from 
%load_ext autoreload
%autoreload 2

# fix system path
import sys
sys.path.append("/home/jovyan/work")

In [3]:
# read in dataset and unzip
df = pd.read_csv('../data/raw/beer_review.zip',compression='zip')

In [4]:
df.head()

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,10325,Vecchio Birraio,1234817823,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986
1,10325,Vecchio Birraio,1235915097,3.0,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon,6.2,48213
2,10325,Vecchio Birraio,1235916604,3.0,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215
3,10325,Vecchio Birraio,1234725145,3.0,3.0,3.5,stcules,German Pilsener,2.5,3.0,Sausa Pils,5.0,47969
4,1075,Caldera Brewing Company,1293735206,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883


In [5]:
df.shape

(1586614, 13)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1586614 entries, 0 to 1586613
Data columns (total 13 columns):
brewery_id            1586614 non-null int64
brewery_name          1586599 non-null object
review_time           1586614 non-null int64
review_overall        1586614 non-null float64
review_aroma          1586614 non-null float64
review_appearance     1586614 non-null float64
review_profilename    1586266 non-null object
beer_style            1586614 non-null object
review_palate         1586614 non-null float64
review_taste          1586614 non-null float64
beer_name             1586614 non-null object
beer_abv              1518829 non-null float64
beer_beerid           1586614 non-null int64
dtypes: float64(6), int64(3), object(4)
memory usage: 157.4+ MB


In [7]:
df.isnull().sum()

brewery_id                0
brewery_name             15
review_time               0
review_overall            0
review_aroma              0
review_appearance         0
review_profilename      348
beer_style                0
review_palate             0
review_taste              0
beer_name                 0
beer_abv              67785
beer_beerid               0
dtype: int64

In [8]:
67785/1586614

0.042723056773733246

Only 4% of the data is missing, since there are already 1.5 mill observations then we can just delete the nulls

In [9]:
# count the number of targets
df.beer_style.value_counts()

American IPA                        117586
American Double / Imperial IPA       85977
American Pale Ale (APA)              63469
Russian Imperial Stout               54129
American Double / Imperial Stout     50705
                                     ...  
Gose                                   686
Faro                                   609
Roggenbier                             466
Kvass                                  297
Happoshu                               241
Name: beer_style, Length: 104, dtype: int64

There are 104 different beer types we need to predict, so our output layer will be of size 104 and the last function will be softmax

In [10]:
df.beer_style.nunique()

104

In [11]:
df.beer_name.value_counts()

90 Minute IPA                          3290
India Pale Ale                         3130
Old Rasputin Russian Imperial Stout    3111
Sierra Nevada Celebration Ale          3000
Two Hearted Ale                        2728
                                       ... 
Viven Bruin                               1
American Hero                             1
Bourbon Barrel-Aged Adoration Ale         1
Hullabaloo                                1
BAB 401                                   1
Name: beer_name, Length: 56857, dtype: int64

In [12]:
df.brewery_name.value_counts()

Boston Beer Company (Samuel Adams)    39444
Dogfish Head Brewery                  33839
Stone Brewing Co.                     33066
Sierra Nevada Brewing Co.             28751
Bell's Brewery, Inc.                  25191
                                      ...  
Panimoravintola Koulu                     1
MonteKristo Beverages                     1
Doshisha Corporation                      1
Karmeliter Bräu                           1
Malmö Brygghus                            1
Name: brewery_name, Length: 5742, dtype: int64

In [13]:
df.review_profilename.value_counts()

northyorksammy    5817
BuckeyeNation     4661
mikesgroove       4617
Thorpe429         3518
womencantsail     3497
                  ... 
hughps               1
andascia             1
spacewolf11          1
dkboileau            1
erosannin            1
Name: review_profilename, Length: 33387, dtype: int64

In [20]:
df.describe()

,brewery_id,review_time,review_overall,review_aroma,review_appearance,review_palate,review_taste,beer_abv,beer_beerid
count,1.586614e+06,1.586614e+06,1.586614e+06,1.586614e+06,1.586614e+06,1.586614e+06,1.586614e+06,1.518829e+06,1.586614e+06
mean,3.130099e+03,1.224089e+09,3.815581e+00,3.735636e+00,3.841642e+00,3.743701e+00,3.792860e+00,7.042387e+00,2.171279e+04
std,5.578104e+03,7.654427e+07,7.206219e-01,6.976167e-01,6.160928e-01,6.822184e-01,7.319696e-01,2.322526e+00,2.181834e+04
min,1.000000e+00,8.406720e+08,0.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e-02,3.000000e+00
25%,1.430000e+02,1.173224e+09,3.500000e+00,3.500000e+00,3.500000e+00,3.500000e+00,3.500000e+00,5.200000e+00,1.717000e+03
50%,4.290000e+02,1.239203e+09,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,6.500000e+00,1.390600e+04
75%,2.372000e+03,1.288568e+09,4.500000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.500000e+00,8.500000e+00,3.944100e+04
max,2.800300e+04,1.326285e+09,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.770000e+01,7.731700e+04


In [32]:
df.describe(include='all')

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
count,1.586614e+06,1586599,1.586614e+06,1.586614e+06,1.586614e+06,1.586614e+06,1586266,1586614,1.586614e+06,1.586614e+06,1586614,1.518829e+06,1.586614e+06
unique,NaN,5742,NaN,NaN,NaN,NaN,33387,104,NaN,NaN,56857,NaN,NaN
top,NaN,Boston Beer Company (Samuel Adams),NaN,NaN,NaN,NaN,northyorksammy,American IPA,NaN,NaN,90 Minute IPA,NaN,NaN
freq,NaN,39444,NaN,NaN,NaN,NaN,5817,117586,NaN,NaN,3290,NaN,NaN
mean,3.130099e+03,NaN,1.224089e+09,3.815581e+00,3.735636e+00,3.841642e+00,NaN,NaN,3.743701e+00,3.792860e+00,NaN,7.042387e+00,2.171279e+04
std,5.578104e+03,NaN,7.654427e+07,7.206219e-01,6.976167e-01,6.160928e-01,NaN,NaN,6.822184e-01,7.319696e-01,NaN,2.322526e+00,2.181834e+04
min,1.000000e+00,NaN,8.406720e+08,0.000000e+00,1.000000e+00,0.000000e+00,NaN,NaN,1.000000e+00,1.000000e+00,NaN,1.000000e-02,3.000000e+00
25%,1.430000e+02,NaN,1.173224e+09,3.500000e+00,3.500000e+00,3.500000e+00,NaN,NaN,3.500000e+00,3.500000e+00,NaN,5.200000e+00,1.717000e+03
50%,4.290000e+02,NaN,1.239203e+09,4.000000e+00,4.000000e+00,4.000000e+00,NaN,NaN,4.000000e+00,4.000000e+00,NaN,6.500000e+00,1.390600e+04
75%,2.372000e+03,NaN,1.288568e+09,4.500000e+00,4.000000e+00,4.000000e+00,NaN,NaN,4.000000e+00,4.500000e+00,NaN,8.500000e+00,3.944100e+04


In [29]:
df.columns

Index(['brewery_id', 'brewery_name', 'review_time', 'review_overall',
       'review_aroma', 'review_appearance', 'review_profilename', 'beer_style',
       'review_palate', 'review_taste', 'beer_name', 'beer_abv',
       'beer_beerid'],
      dtype='object')

Columns to remove:
- Brewery_id: Use this as the label encoder?
- review_time: not relevant
- beer_name: very indicative of beer style, not to be included
- review_profilename: not relevant
- review_overall: not sure yet
- beer_abv: very indicative of beer style
- beer_beerid: not needed

Have to include
- brewery_name
- review_aroma
- review_appearance
- review_palate
- review_taste

Target
- beer_style


# Create datasets

In [40]:
df_cleaned = df.copy()

In [41]:
df_cleaned.drop(['brewery_id','review_time','beer_name','review_profilename','review_overall','beer_abv','beer_beerid'],axis=1,inplace=True)

In [42]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1586614 entries, 0 to 1586613
Data columns (total 6 columns):
brewery_name         1586599 non-null object
review_aroma         1586614 non-null float64
review_appearance    1586614 non-null float64
beer_style           1586614 non-null object
review_palate        1586614 non-null float64
review_taste         1586614 non-null float64
dtypes: float64(4), object(2)
memory usage: 72.6+ MB


In [43]:
df_cleaned.isnull().sum()

brewery_name         15
review_aroma          0
review_appearance     0
beer_style            0
review_palate         0
review_taste          0
dtype: int64

In [44]:
df_cleaned.dropna(inplace=True)

In [45]:
df_cleaned.isnull().sum()

brewery_name         0
review_aroma         0
review_appearance    0
beer_style           0
review_palate        0
review_taste         0
dtype: int64

In [46]:
# need to use drop=True to drop the index column created
df_cleaned.reset_index(drop=True,inplace=True)

In [47]:
df_cleaned.head()

,brewery_name,review_aroma,review_appearance,beer_style,review_palate,review_taste
0,Vecchio Birraio,2.0,2.5,Hefeweizen,1.5,1.5
1,Vecchio Birraio,2.5,3.0,English Strong Ale,3.0,3.0
2,Vecchio Birraio,2.5,3.0,Foreign / Export Stout,3.0,3.0
3,Vecchio Birraio,3.0,3.5,German Pilsener,2.5,3.0
4,Caldera Brewing Company,4.5,4.0,American Double / Imperial IPA,4.0,4.5


In [48]:
df_cleaned.describe(include='all')

,brewery_name,review_aroma,review_appearance,beer_style,review_palate,review_taste
count,1586599,1.586599e+06,1.586599e+06,1586599,1.586599e+06,1.586599e+06
unique,5742,NaN,NaN,104,NaN,NaN
top,Boston Beer Company (Samuel Adams),NaN,NaN,American IPA,NaN,NaN
freq,39444,NaN,NaN,117584,NaN,NaN
mean,NaN,3.735638e+00,3.841647e+00,NaN,3.743705e+00,3.792864e+00
std,NaN,6.976142e-01,6.160899e-01,NaN,6.822131e-01,7.319658e-01
min,NaN,1.000000e+00,0.000000e+00,NaN,1.000000e+00,1.000000e+00
25%,NaN,3.500000e+00,3.500000e+00,NaN,3.500000e+00,3.500000e+00
50%,NaN,4.000000e+00,4.000000e+00,NaN,4.000000e+00,4.000000e+00
75%,NaN,4.000000e+00,4.000000e+00,NaN,4.000000e+00,4.500000e+00


In [49]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1586599 entries, 0 to 1586598
Data columns (total 6 columns):
brewery_name         1586599 non-null object
review_aroma         1586599 non-null float64
review_appearance    1586599 non-null float64
beer_style           1586599 non-null object
review_palate        1586599 non-null float64
review_taste         1586599 non-null float64
dtypes: float64(4), object(2)
memory usage: 72.6+ MB


# Encode categorical variables brewery_name and beer_style (target)

https://yashuseth.blog/2018/07/22/pytorch-neural-network-for-tabular-data-with-categorical-embeddings/

https://www.fast.ai/2018/04/29/categorical-embeddings/

https://www.kaggle.com/vadbeg/pytorch-nn-with-embeddings-and-catboost/notebook#Features-encoding

https://stackabuse.com/introduction-to-pytorch-for-classification/

In [50]:
from sklearn.preprocessing import LabelEncoder

In [53]:
# instantiate label encoder
LE = LabelEncoder()

In [54]:
cat_cols = ['brewery_name','beer_style']

In [56]:
# fit transform cat variables - 

label_encoders = {}
for cat_col in cat_cols:
        label_encoders[cat_col] = LE
        df_cleaned[cat_col] = label_encoders[cat_col].fit_transform(df_cleaned[cat_col])

In [57]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1586599 entries, 0 to 1586598
Data columns (total 6 columns):
brewery_name         1586599 non-null int64
review_aroma         1586599 non-null float64
review_appearance    1586599 non-null float64
beer_style           1586599 non-null int64
review_palate        1586599 non-null float64
review_taste         1586599 non-null float64
dtypes: float64(4), int64(2)
memory usage: 72.6 MB


In [59]:
df_cleaned[['brewery_name','beer_style']].describe()

,brewery_name,beer_style
count,1.586599e+06,1.586599e+06
mean,2.905176e+03,4.214239e+01
std,1.729725e+03,3.302083e+01
min,0.000000e+00,0.000000e+00
25%,1.325000e+03,1.200000e+01
50%,2.880000e+03,3.100000e+01
75%,4.581000e+03,7.400000e+01
max,5.741000e+03,1.030000e+02


In [60]:
label_encoders

{'brewery_name': LabelEncoder(), 'beer_style': LabelEncoder()}

In [ ]:
# create a function for pipepline later on - 
#https://scikit-learn.org/stable/auto_examples/compose/plot_column_transformer.html#sphx-glr-auto-examples-compose-plot-column-transformer-py

from sklearn.preprocessing import FunctionTransformer

def cat_label_encoder(data,cat_cols):
    
    from sklearn.preprocessing import LabelEncoder
    LE = LabelEncoder()
    # create an empty dict to keep label encoder for each cat col
    label_encoders = {}
    # loop over cat_cols and encode
    for cat_col in cat_cols:
        label_encoders[cat_col] = LE
        data[cat_col] = label_encoders[cat_col].fit_transform(data[cat_col])
        
    return data

cat_label_transformer = FunctionTransformer(cat_label_encoder)

In [61]:
# Need to StandardScaler the numeric columns
num_cols = ['review_aroma','review_appearance','review_palate','review_taste']

In [62]:
# instantiate scaler
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

In [63]:
df_cleaned[num_cols] = sc.fit_transform(df_cleaned[num_cols])

In [64]:
df_cleaned.head()

,brewery_name,review_aroma,review_appearance,beer_style,review_palate,review_taste
0,5438,-2.487964,-2.177682,65,-3.288863,-3.132476
1,5438,-1.771235,-1.366111,51,-1.090136,-1.083199
2,5438,-1.771235,-1.366111,59,-1.090136,-1.083199
3,5438,-1.054506,-0.554541,61,-1.823045,-1.083199
4,1480,1.095680,0.257029,9,0.375682,0.966078


In [65]:
from src.data.sets import split_sets_random

In [66]:
X_train, y_train, X_val, y_val, X_test, y_test = split_sets_random(df_cleaned, target_col='beer_style', test_ratio=0.2, to_numpy=True)

In [68]:
print(X_train.shape)
print(y_train.shape)

(951959, 5)
(951959,)


In [69]:
print(X_val.shape)
print(X_test.shape)

(317320, 5)
(317320, 5)


In [75]:
from src.data.sets import save_sets

save_sets(X_train=X_train, y_train=y_train, X_val=X_val, y_val=y_val, X_test=X_test, y_test=y_test)

In [71]:
X_val

array([[ 4.88400000e+03,  3.78951241e-01,  2.57028931e-01,
         3.75681950e-01,  2.82985337e-01],
       [ 1.63000000e+02, -1.05450621e+00, -5.54541255e-01,
        -1.09013615e+00, -1.08319920e+00],
       [ 2.17000000e+02, -3.37777485e-01,  2.57028931e-01,
        -3.57227099e-01,  2.82985337e-01],
       ...,
       [ 1.26600000e+03,  3.78951241e-01,  1.06859912e+00,
         3.75681950e-01,  2.82985337e-01],
       [ 3.58000000e+03,  3.78951241e-01,  2.57028931e-01,
         1.10859100e+00,  1.64916987e+00],
       [ 4.70900000e+03, -1.05450621e+00,  1.06859912e+00,
        -3.57227099e-01, -4.00106932e-01]])

In [73]:
pwd()

'/home/jovyan/work/notebooks'

In [76]:
# need to know which are cat cols and which are num cols
categorical = df_cleaned.drop(['beer_style'] + num_cols,
                          axis=1).columns


In [77]:
categorical

Index(['brewery_name'], dtype='object')

In [80]:
cat_cols_idx, cont_cols_idx = list(), list()

for idx, column in enumerate(df_cleaned.drop('beer_style',
                                         axis=1).columns):
    if column in categorical:
        cat_cols_idx.append(idx)
    elif column in num_cols:
        cont_cols_idx.append(idx)

In [81]:
print(cat_cols_idx)
print(cont_cols_idx)

[0]
[1, 2, 3, 4]


In [87]:
cat_dim = [int(df_cleaned[col].nunique()) for col in cat_cols]
cat_dim = [[x, min(200, (x + 1) // 2)] for x in cat_dim]

for el in cat_dim:
    if el[0] < 10:
        el[1] = el[0]

cat_dim


[[5742, 200], [104, 52]]

In [88]:
no_of_embs = sum([y for x, y in cat_dim])

In [89]:
no_of_embs

252